In [2]:
import cPickle as pickle
import os, sys, random
sys.path.append('/Users/matt.meng/dev/seq2seq_model')

In [2]:
import pandas as pd
from data_preprocess import tokenize_title_column, \
process_title_column_by_spacy, create_selected_vocab_dict, process_title_with_token_dict, create_crambled_training

### the title processing steps

In [3]:
meta_data_file_name = 'meta_title_data.csv'
delimiter = '\t\t'
data_path = '/Users/matt.meng'

In [4]:
data = pd.read_csv(os.path.join(data_path, meta_data_file_name), index_col='url', delimiter=delimiter, encoding='utf-8')

data.dropna(how='any', inplace=True)
data['publisherId'] = data['publisherId'].astype(int).astype(str)
valid_publisher_ids = ['1001082', '1023406', '1003264', '1040522', '782', '1006541',
                           '1168', '1038583', '1021516', '580', '1020689', '1031851', '1001264',
                           '1039208', '1054980', '1018671', '1031841', '1031842', '1031852',
                           '1008941', '1003764', '1068057', '1038711', '1002628', '1031853',
                           '1021578', '1043813', '1010748', '1040526', '1005092', '612',
                           '1003870', '1001156', '1012083', '1017946', '1041479', '1027016',
                           '1010488', '1017947', '1010497', '1038582', '1045821', '1020968',
                           '1037842', '1029984', '723', '196', '1030941']

filtered_data = data.loc[data['publisherId'].isin(valid_publisher_ids), :]
unique_filtered_data = filtered_data[~filtered_data.index.duplicated(keep='first')]
print unique_filtered_data.shape

processed_column_name = 'processed_title'
pageView_column_name = 'traffic'
filtered_data = tokenize_title_column(unique_filtered_data, processed_column_name, pageView_column_name)

    #all_titles, vocab_dict = process_title_column(filtered_data, 'processed_title', 'traffic')
all_titles, vocab_dict = process_title_column_by_spacy(filtered_data, 'processed_title', 'traffic', skip_stop_words=True)

/Users/matt.meng/.virtualenvs/kaggle/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


(128155, 3)


/Users/matt.meng/dev/seq2seq_model/data_preprocess.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['title_word_counts'], data[processed_column_name] = zip(*data[title_column_name].map(basic_tokenizer))


finish the tokenization...
finish processing 10000 titles with spaCy, using 8.29 seconds
finish processing 20000 titles with spaCy, using 14.19 seconds
finish processing 30000 titles with spaCy, using 20.48 seconds
finish processing 40000 titles with spaCy, using 28.20 seconds
finish processing 50000 titles with spaCy, using 35.06 seconds
finish processing 60000 titles with spaCy, using 43.13 seconds
finish processing 70000 titles with spaCy, using 52.38 seconds
finish processing 80000 titles with spaCy, using 61.76 seconds
finish processing 90000 titles with spaCy, using 70.30 seconds
finish processing 100000 titles with spaCy, using 80.57 seconds
finish processing 110000 titles with spaCy, using 92.72 seconds
total 29594 tokens are identified...


#### check the `vocab_ditc` content

In [ ]:
sorted_pairs = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)

In [ ]:
sorted_pairs

### create the `token_dict` and the scrambled data

In [ ]:
UKN_index = len(TOKEN_DICT) - 1
token_dict, reverse_token_dict = create_selected_vocab_dict(vocab_dict, UKN_index, token_freq_threshold=4)
selected_content = process_title_with_token_dict(all_titles, token_dict, reverse_token_dict, UKN_index, UKN_frac_threshold=0.2)
processed_content = create_crambled_training(selected_content)

### load the processed pickle and check the content

In [3]:
data_path = '/Users/matt.meng'
#pickle_file = 'processed_titles_data.pkl'
pickle_file = 'lemmanized_no_stop_words_CBOW_data.pkl'
with open(os.path.join(data_path, pickle_file), 'r') as input_file:
     data = pickle.load(input_file)

In [4]:
data.keys()

['reverse_token_dict', 'training_list', 'target_list', 'token_dict']

In [5]:
training_list = data['training_list']
target_list = data['target_list']

In [6]:
print len(training_list)

553544


In [7]:
training_list[:4]

[[138, 109], [11821, 161], [2425, 279], [872, 28]]

In [ ]:
titles = data['titles']
token_dict = data['token_dict']
reverse_token_dict = data['reverse_token_dict']

In [ ]:
token_dict.keys()[:20]

In [ ]:
index = 50
map(reverse_token_dict.get, titles[index])

In [ ]:
'''
training_titles = data['training_titles']
target_titles = data['target_titles']

index = 20000
print training_titles[index]
print target_titles[index]
'''

### validate the DataGenerator with dual outputs

In [ ]:
from data_preprocess import TOKEN_DICT, _GO, _EOS
from data import DataGenerator, process_batch

In [ ]:
pickle_file = 'scramble_titles_data.pkl'
batch_size = 32
pickle_file_path = os.path.join(os.path.expanduser("~"), pickle_file)

## the dual_output test
#dataGen = DataGenerator(pickle_file_path)
dataGen = DataGenerator(pickle_file_path, True)
batches = dataGen.generate_sequence(batch_size)
training_batch, target_batch = next(batches)
  

In [ ]:
training_batch

In [ ]:
target_batch

In [ ]:
encoder_inputs_, encoder_inputs_length = process_batch([sequence + [TOKEN_DICT[_EOS]] for sequence in training_batch])
decoder_inputs_, decoder_inputs_length = process_batch([[TOKEN_DICT[_GO]] + sequence for sequence in target_batch])


In [ ]:
encoder_inputs_

In [ ]:
decoder_inputs_